In [2]:
import json

In [7]:
with open("data/train/00001756c60be8.json",'r') as f: 
   jsondict = json.load(f)  

# structure
Here I check the structure of the JSON, which consist of source and cell_type, two dictionaries containing the content and cell type (markdown or code) based on hash

In [10]:
print(jsondict["source"]['1862f0a6'])

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [14]:
print(jsondict['cell_type']['1862f0a6'])

code


# Reconstructing iPython Notebook from the dateset

Unfortunately it is hard to understand this data. The most straightforward way to visualize them is in fact to reconstruct the raw data, aka the original notebooks 

In [53]:
# make function to add code and comment cells to the notebook template using python f-strings

def makepynb2(cells = None):
    pynb_template ={
    "cells": [] if None else cells,
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3 (ipykernel)",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.9.9"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 5
}

    return pynb_template


In [61]:
print(makepynb2("hey there"))

{'cells': 'hey there', 'metadata': {'kernelspec': {'display_name': 'Python 3 (ipykernel)', 'language': 'python', 'name': 'python3'}, 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3}, 'file_extension': '.py', 'mimetype': 'text/x-python', 'name': 'python', 'nbconvert_exporter': 'python', 'pygments_lexer': 'ipython3', 'version': '3.9.9'}}, 'nbformat': 4, 'nbformat_minor': 5}


In [63]:
def makebox2(ctype = "code", hash = "12345", content = ""):
    if ctype == "markdown":
        celldict =        {
            "cell_type": "markdown",
            "id": hash,
            "metadata": {},
            "source": [content]
        }
    if ctype == "code":
        celldict = \
        {
            "cell_type": "code",
            "execution_count": 1,
            "id": hash,
            "metadata": {},
            "outputs": [],
            "source": [content]
        }   
    return celldict

In [64]:
makebox2()

{'cell_type': 'code',
 'execution_count': 1,
 'id': '12345',
 'metadata': {},
 'outputs': [],
 'source': ['']}

In [38]:
filehash = '00001756c60be8'
cellhashes = "1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 77e56113 2eefe0ef 1ae087ab 0beab1cd 8ffe0b25 9a78ab76 0d136e08 8a4c95d1 23705731 ebe125d5 aaad8355 d9dced8b 21616367 86497fe1 c3ce0945 e2c8e725 a6357f7e ff7c44ed ac301a84 0e7c906e dd0c804a 45082c89 781bbf3c 4bb2e30a bd94f005 63c26fa2 62638fba 3e5f860d bb69e88c 6b5664c7 3eebeb87 23783525 36002912 bfbde93e 8522781a 1496beaf 8ca8392c b69a4f9b 17ec3fc4 503926eb 76512d50 032e2820 a98c5d9f 06365725 8554b284 59959af5 2e1a5949 80151ab7 fcb6792d 5bf9ca51 915643b3 f5504853 9f50dca0"
cellhashlist = cellhashes.split()

In [58]:
celltext = []
for cellhash in cellhashlist:
    celltext.append( makebox2(ctype = jsondict['cell_type'][cellhash],
                        hash = cellhash,
                        content = jsondict["source"][cellhash]))
pynb = makepynb2(celltext)

In [59]:
with open("visualize_automate.ipynb",'w') as fw:
    json.dump(pynb, fw)

In [60]:
with open("visualize_automate.ipynb",'r') as f: 
   pynbjson = json.load(f)  

In [49]:
jsondict["source"]['1862f0a6']

'# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session'

# Applying to all json files

Now we apply the functions to all files

In [66]:
import os

In [68]:
def convert_to_ipynb(filehash, cellhashlist):
    # open json from dataset
    with open(f"data/train/{filehash}.json",'r') as f: 
        jsondict = json.load(f)  

    # manipulate
    celltext = []
    for cellhash in cellhashlist:
        celltext.append( makebox2(ctype = jsondict['cell_type'][cellhash],
                            hash = cellhash,
                            content = jsondict["source"][cellhash]))
    pynb = makepynb2(celltext)

    #write to destination
    with open(f"visualize/{filehash}.ipynb",'w') as fw:
        json.dump(pynb, fw)

139256

In [73]:
for f in os.listdir("data/train" ):
    hash = f.split(".")[0]
    convert_to_ipynb(hash, )

139256

In [74]:
import pandas

In [90]:
train_order = pandas.read_csv("data/train_orders.csv")
train_order

,id,cell_order
0,00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 7...
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...
2,0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 0...
3,0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 d...
4,0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a...
...,...,...
139251,fffc30d5a0bc46,09727c0c ff1ea6a0 ddfef603 a01ce9b3 3ba953ee b...
139252,fffc3b44869198,978a5137 faa48f03 28dfb12a eea2e812 64fef97c 4...
139253,fffc63ff750064,5015c300 411b85d9 8238198c f4781d1d b5532930 e...
139254,fffcd063cda949,7e6266ad d8281fc5 d4ffcaef 3e0e4a47 21387fc8 c...


In [91]:
train_order.cell_order=train_order.cell_order.str.split()

Too many files to output, we will only do one in one hundred

In [96]:
%% time
for index, row in train_order.iterrows():
    if index%100==0:
        #print(row["id"])
        #print(row["cell_order"])
        convert_to_ipynb(row["id"], row["cell_order"] )

In [98]:
101%100

1